In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances, explained_variance_score
import plotly.express as px
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm

In [ ]:
data = pd.read_csv('data/processed_final.gz')
print(data.shape)
data['nothing'] = 0.0
data.head()

In [ ]:
env_features = ['stratum', 'depth_m', 'duration_hr', 'surface_temperature_c', 'bottom_temperature_c']
indexes = ['year', 'srvy', 'station', 'haul']
species = sorted([c for c in data.columns if c not in (indexes + env_features + ['nothing'])])

# Abundance

In [ ]:
data_train, data_test, *_ = train_test_split(data, data, test_size=0.2, random_state=42)

In [ ]:
env_features = ['stratum', 'depth_m', 'duration_hr', 'surface_temperature_c', 'bottom_temperature_c']
indexes = ['year', 'srvy', 'station', 'haul']
species = sorted([c for c in data.columns if c not in (indexes + env_features + ['nothing'])])

In [ ]:
X = np.array([
    data[s] ** (1/4) / np.max(data[s] ** (1/4)) for s in species
])
distance_matrix = pairwise_distances(X, metric='braycurtis')

In [ ]:
N_CLUSTERS = 6

model = AgglomerativeClustering(
    n_clusters=N_CLUSTERS,
    metric='precomputed',
    linkage='complete',
    compute_distances=True
).fit(distance_matrix)

In [ ]:
def train_forest(
    data_train, data_test, selected, species_features, env_features
):
    data_train_resample = (
        data_train[data_train[selected] > 0]
        .sample(frac=1, random_state=42).reset_index(drop=True)
    )
    data_test_resample = (
        data_test[data_test[selected] > 0]
        .sample(frac=1, random_state=42).reset_index(drop=True)
    )

    X_train = data_train_resample[env_features + species_features]
    y_train = data_train_resample[selected]

    X_test = data_test_resample[env_features + species_features]
    y_test = data_test_resample[selected]

    forest = RandomForestRegressor(
        n_estimators=100,
        min_samples_leaf=5,
        random_state=42,
        n_jobs=-1,
    )
    search = RandomizedSearchCV(
        forest,
        {
            'min_samples_leaf': [5, 10, 20],
        },
        n_iter=3,
        refit=True,
        cv=3,
        verbose=0
    )
    search.fit(X_train, y_train)
    forest = search.best_estimator_
    y_pred = forest.predict(X_test)
    score = explained_variance_score(y_test, y_pred)
    return score

for case, features in zip(['cluster_env', 'cluster_species', 'rand_env', 'rand_species'], [env_features, ['nothing'], env_features, ['nothing']]):
    print(f'Case: {case}')
    selection_rows = []
    for i in range(N_CLUSTERS):
        left_to_select = list(str(s) for s in np.array(species)[model.labels_ == i])
        if case.startswith('rand'):
            left_to_select = [str(s) for s in np.random.choice(species, size=len(left_to_select), replace=False)]

        for s in left_to_select:
            data_train[s] = data_train[s] ** (1/4)
            data_test[s] = data_test[s] ** (1/4)

        selected = []
        scores = []
        selected_scores = []
        
        information = 1.0
        marginal_information = 1.0
        for j in tqdm(range(len(left_to_select))):
            options = []
            for new_selection in left_to_select:
                species_features = [s for s in left_to_select if s != new_selection]
                overall_score = (len(left_to_select) - 1)
                selected_score = 0.0
                for s in selected + [new_selection]:
                    score = train_forest(
                        data_train, data_test, s, species_features, features
                    )
                    overall_score += score
                    selected_score += score
                overall_score = (overall_score / (len(selected) + len(left_to_select)))
                selected_score = selected_score / (len(selected) + 1)
                options.append((new_selection, overall_score, selected_score))
            best_option = sorted(options, key=lambda x: x[1], reverse=True)[0]

            selection_rows.append({
                'case': case,
                'cluster': i,
                'species': best_option[0],
                'information': float(information),
                'marginal': float(marginal_information),
                'species': len(left_to_select) + len(selected) - j,
            })
            information = best_option[1]
            marginal_information = best_option[2]

            left_to_select.remove(best_option[0])
            selected.append(best_option[0])

        selection_rows.append({
            'case': case,
            'cluster': i,
            'species': 'BASE',
            'information': float(information),
            'marginal': float(marginal_information),
            'species': 0,
        })
        break

    selection_df = pd.DataFrame(selection_rows)
    selection_df.to_csv(f'abundance_{case}.csv', index=False)
